<a href="https://colab.research.google.com/github/futartup/S8-assignment/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import all the custom and inbuilt modules

In [0]:
try:
  import ipynb
except:
  !pip install ipynb --upgrade
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import os, sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/S8')
from torchsummary import summary
import torch.optim as optim
import torch.nn as nn
import torchvision

# custom apis
from ipynb.fs.full.transform_train_test_loader import *
from ipynb.fs.full.conf import *
from ipynb.fs.full.test import *
from ipynb.fs.full.train import *

# The model related code. Load the model from the conf file. 

In [3]:

if custom_model['use']:
  #use custom model
  pass
elif inbuilt_model['use']:
  from ipynb.fs.full.model.pytorch_resnet import *
  from ipynb.fs.full.model.resnet import *
  from ipynb.fs.full.model.QuizDNN import *
  model_type = inbuilt_model['model']
  #net = globals()[model_type]()
  net = torchvision.models.resnet18(pretrained=False)

torch.Size([1, 10])
Current Date/Time:  2020-03-17 16:53:15.644992


# Model Summary

In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = net.to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

train_loader = get_train_loader()
test_loader = get_test_loader()
    
# TODO: Take the end epoch value from conf file 
for e in range(1, 60):
    print("epoch no: " , e)
    train_model(model, device, train_loader, optimizer, criterion, e)
    test_model(model, device, test_loader, criterion)
    print("===========================================================\n")



Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/782 [00:00<?, ?it/s]

epoch no:  1


Loss=2.1946065425872803 Batch_id=781 Accuracy=33.97: 100%|██████████| 782/782 [00:34<00:00, 22.83it/s]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.0243, Accuracy: 4312/10000 (43.12%)


epoch no:  2


...


Test set: Average loss: 0.0098, Accuracy: 8014/10000 (83.47%)


epoch no:  58


Loss=0.4113151431083679 Batch_id=781 Accuracy=87.65: 100%|██████████| 782/782 [00:35<00:00, 21.90it/s]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.0091, Accuracy: 8084/10000 (85.41%)


epoch no:  59


Loss=0.1510634422302246 Batch_id=781 Accuracy=87.82: 100%|██████████| 782/782 [00:35<00:00, 21.90it/s]
100%|██████████| 157/157 [00:02<00:00, 54.19it/s]


Test set: Average loss: 0.0093, Accuracy: 8024/10000 (85.13%)


